In [ ]:
# ==========================
# 📦 Import Required Libraries
# ==========================
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib

# ==========================
# ⚙️ Configuration Settings
# ==========================
SENSOR_HZ = 67  # Sampling rate of sensor in Hz
TIME_STEPS = int(SENSOR_HZ * 0.75)  # Window length (0.75s of data)
if TIME_STEPS == 0:
    TIME_STEPS = 1

N_FEATURES = 6  # Number of sensor features: accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z

# ==========================
# 🧠 Model Architecture
# ==========================
def build_simple_model(time_steps, n_features, num_classes):
    """
    Build a simple 1D CNN for time-series classification.
    """
    model = Sequential([
        Input(shape=(time_steps, n_features)),

        # First convolution layer
        Conv1D(filters=16, kernel_size=5, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),

        # Second convolution layer
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),

        # Fully connected layers
        Flatten(),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')  # Output layer
    ])
    return model

# ==========================
# 📈 Training Function
# ==========================
def train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
    """
    Compile and train the model using early stopping.
    """
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )
    return history

# ==========================
# 🚀 Main Execution
# ==========================
if __name__ == "__main__":
    # --- Load CSV Dataset ---
    try:
        df = pd.read_csv('gesture_datas1.csv')
    except FileNotFoundError:
        print("Error: 'gesture_datas1.csv' not found. Please ensure the dataset is in the same directory.")
        exit()

    # --- Extract Features and Labels ---
    sensor_features = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
    X = df[sensor_features].values
    y = df['gesture_label'].values

    # --- Standardize Features ---
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # --- Create Time-Series Sequences using Sliding Window ---
    X_seq, y_seq = [], []
    for label in np.unique(y):
        label_indices = np.where(y == label)[0]
        label_data = X_scaled[label_indices]

        for i in range(0, len(label_data) - TIME_STEPS + 1, 1):
            X_seq.append(label_data[i:i + TIME_STEPS])
            y_seq.append(label)

    X_seq = np.array(X_seq)
    y_seq = np.array(y_seq)

    # --- Encode Labels ---
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_seq)
    NUM_CLASSES = len(label_encoder.classes_)
    y_one_hot = tf.keras.utils.to_categorical(y_encoded, num_classes=NUM_CLASSES)

    # --- Summary Info ---
    print(f"Calculated TIME_STEPS: {TIME_STEPS}")
    print(f"Detected classes: {label_encoder.classes_}")
    print(f"Number of classes: {NUM_CLASSES}")

    # --- Handle Edge Case: No Data ---
    if X_seq.size == 0 or y_one_hot.size == 0:
        print("Error: No sequences generated. Check TIME_STEPS and dataset size. Make sure 'gesture_datas1.csv' contains sufficient sensor data and a 'gesture_label' column.")
        exit()

    # --- Train-Test Split ---
    X_train, X_val, y_train, y_val = train_test_split(
        X_seq, y_one_hot,
        test_size=0.2,
        random_state=256,
        stratify=y_one_hot
    )

    # --- Build Model ---
    model = build_simple_model(TIME_STEPS, N_FEATURES, NUM_CLASSES)
    model.summary()

    # --- Train Model ---
    history = train_model(model, X_train, y_train, X_val, y_val)

    # --- Evaluate Model ---
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Loss: {loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.4f}")

    # --- Save Model and Preprocessing Artifacts ---
    model.save('simple_gesture_recognition_model_1DQ.h5')
    joblib.dump(label_encoder, 'simple_label_encoderQ.pkl')
    joblib.dump(scaler, 'simple_scaler_1Dq.pkl')
    print("Model, label encoder, and scaler saved.")


In [1]:
import tensorflow as tf

# --- Input and output paths ---
h5_model_path = 'simple_gesture_recognition_model_1DQ.h5'  # Replace with your actual file
tflite_model_path = 'converted_model.tflite'

# --- Load the .h5 Keras model ---
model = tf.keras.models.load_model(h5_model_path)

# --- Convert to TensorFlow Lite format ---
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# --- Save the .tflite model ---
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print("[INFO] Conversion complete:", tflite_model_path)


c:\Users\devda\OneDrive\Desktop\ML couse Practice\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\devda\OneDrive\Desktop\ML couse Practice\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\devda\OneDrive\Desktop\ML couse Practice\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

INFO:tensorflow:Assets written to: C:\Users\devda\AppData\Local\Temp\tmp3x3okzgb\assets


INFO:tensorflow:Assets written to: C:\Users\devda\AppData\Local\Temp\tmp3x3okzgb\assets


Saved artifact at 'C:\Users\devda\AppData\Local\Temp\tmp3x3okzgb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 6), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2639048999760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639049001104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639049001488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639049002256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639049002832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639049003792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639049004368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2639049005328: TensorSpec(shape=(), dtype=tf.resource, name=None)
[INFO] Conversion complete: converted_model.tflite
